# 호텔 챗봇

이 노트북에서는 호텔 챗봇 문제를 다룹니다. 어떻게 하면 호텔 프론트 데스크에서 더 나은 고객 서비스를 제공할 수 있을까요? 간단한 방법 중 하나는 호텔에서 경험할 수 있는 일들에 관한 간단한 질문에 대답할 수 있는 FAQ 채팅 로봇을 갖는 것입니다. 챗봇을 갖는 것에는 많은 장점이 있습니다.

1. 호텔의 매력을 높이고 정보 처리량을 늘립니다.
2. 호텔에 대한 질문을 데이터 테이블 형식으로 수집할 수 있는 방법을 만듭니다.

이 노트북에서는 두 가지 모델을 살펴볼 것입니다. 코사인 유사성 및 doc2vec 모델입니다.


## 1. 챗봇에 지식 기반 추가

챗봇의 대화 능력은 사용할 수 있는 데이터에 의해 정의됩니다. Ques.txt 파일과 ans.txt 파일에서 질문과 답변을 살펴보십시오. 이 챗봇은 기본적으로 질문에 대하여 질문 은행과 코사인 유사성을 확인하여 답을 찾으려고 노력할 것입니다.

# 챗봇만들기 프로세스

챗봇만들기
1. 라이브러리 불러오기
2. 데이터파일 불러오기 
3. 데이터전처리
- 문장토큰화
- 소문자 변경
- 단어토큰화
- 표제어추출
4. 챗봇만들기
코드 작성 

### 1.1 라이브러리 가져오기

In [1]:
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random 
import operator
import string # 표준 파이썬 문자열을 처리
import re
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

C:\Users\isfs0\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#강사님 코드
import os
# 단계 1: 파일 읽기 및 저장 (저장시 소문자로 변경하여 저장하기 )
ques_filepath = os.getcwd()+r'//[Dataset] Module27(ques).txt'
ans_filepath = os.getcwd()+r'//[Dataset] Module27 (ans).txt'

# 파일 내용을 문자열로 읽기
with open(ques_filepath, 'r', errors='ignore') as file:
    raw_data_ques = file.read()
with open(ans_filepath, 'r', errors='ignore') as file:
    raw_data_ans = file.read()
raw_data_ans

"200$ per night is the price for a basic suite.\n\nThis establishment was constructed and inaugurated by John S. on the 23rd of September 1965.\n\nBreakfast is served from 7 AM to 10 AM.\n\nThe breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before. \n\nThe Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.\n\nThere are 43 rooms in this hotel including one pent house suite.\n\nWe offer 4 types of rooms: basic, mid-level, premium and penthouse.\n\nThis hotel is called Vance Hotel.\n\nYes, room service is available 24 hrs.\n\nTo call room service, please dial '0' using the phone in your room.\n\nYes we have one restaurant currently called 'Rouge'.\n\nThere are 12 floors in the hotel.\n\n300$ per night is the standard price for a mid-level suite.\n\n500$ per night is the price fo

In [3]:
   
# 문장 토큰화 수행
sent_tokens_ques = nltk.sent_tokenize(raw_data_ques)
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)

print(len(sent_tokens_ques), len(sent_tokens_ans))

47 47


In [4]:
# 소문자로 변경하기
sent_tokens_ans = [sentence.lower() for sentence in sent_tokens_ans]
sent_tokens_ques = [sentence.lower() for sentence in sent_tokens_ques]
print(len(sent_tokens_ques), len(sent_tokens_ans))

# 질문과 답변 매칭한 딕셔너리 - ques_and_pairs생성합니다. 
ques_and_pairs = {}
ques_and_pairs = {sent_tokens_ques[i]: sent_tokens_ans[i] 
                                for i in range(len(sent_tokens_ans))} 

47 47


단어토큰화

In [5]:
word_tokens_ques = nltk.word_tokenize(raw_data_ques)
raw_data_ques

'What is the price of one night stay in basic suite?\n\nHow old is this establishment?\n\nWhat time is breakfast served?\n\nWhat is the breakfast menu?\n\nWhat is the history behind this hotel or establishment?\n\nHow many rooms are there in this hotel?\n\nWhat are the types of rooms or suites offered by the hotel?\n\nWhat is the name of the hotel?\n\nIs room service served 24 hours?\n\nHow do I call for room service?\n\nAre there any restaurants in the hotel?\n\nHow many floors are there in the hotel?\n\nWhat is the price of one night stay at the mid-level suite?\n\nWhat is the price of one night stay at the premium suite?\n\nDo you have tuxedo services?\n\nDo you have a laundry service?\n\nWhat time do the restaurants open for dinner?\n\nWhat are the near by tourist attractions?\n\nIs there a spa in the hotel?\n\nIs there anywhere I can get a massage?\n\nWhat time is the check in?\n\nWhat time is the check out?\n\nDo you offer handicapped rooms?\n\nIs parking available at the hotel?\

In [11]:
# 표제어추출

In [6]:
def LemNormalize(tokens):
    lemmer = nltk.stem.WordNetLemmatizer()
    return [lemmer.lemmatize(token) for token in tokens]
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

In [11]:
# GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
# GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [7]:
def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

In [8]:
def response(user_response):
    
    robo_response='' # 문자열을 포함하도록 변수를 초기화
    sent_tokens_ques.append(user_response) # sent_messages에 사용자 응답 추가
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens_ques) # tfidf 값 가져오기
    vals = cosine_similarity(tfidf[-1], tfidf) # 코사인 유사성 값 가져오기
    idx=vals.argsort()[0][-2] 
    
    flat = vals.flatten() 
    flat.sort() # 오름차순으로 정렬
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response, vals
    else:
        robo_response = robo_response+ques_and_pairs[sent_tokens_ques[idx]]
        return robo_response, vals
        
# 2.1.4 챗봇 테스트
flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))
            else:
                print("Jane: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens_ques.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')
    else:
        flag=False
        print("Jane: Bye! take care..")

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
ㅠㅛㄷ


NameError: name 'GREETING_INPUTS' is not defined

### 1.2 데이터 처리
#### 1.2.1 파일 읽기 및 저장

In [24]:
# 1.[Dataset] Module27 (ans).txt 파일을 읽어 raw_data_ans 에 저장한다.
# 2.[Dataset] Module27 (ques).txt 파일을 읽어 raw_data_ques 에 저장한다. 
# 단계 1: 파일 경로 지정하기읽기 및 저장 : ques_filepath, ans_filepath

# your code

- raw_data_ans와 raw_data_ques의 길이 확인(For Mapping)

In [25]:
# 소문자 처리 answer & Question
# your code

In [26]:
# 저장된 데이터의 길이 확인
# your code here


In [27]:
# 저장된 데이터의 길이 확인
# your code here


# 1.2.2 소문자처리

In [28]:
from nltk.tokenize import sent_tokenize

# your code

####  세분화, 표제어 추출,  단어 토큰화

모듈 26 코사인 유사성 노트북에서 배운 방법을 참조하여 데이터를 세분화, 토큰화, 표제어 추출로 전처리 진행합니다.

#### 1.2.3 문장 세분화 

In [29]:
# 1.2.3 문장 세분화 후 길이확인하기
# question 과 answer 데이터에 대해 sent_tokenize() 사용하여 문서를 문장 목록으로 변환한다 
# sent_tokens_ques / sent_tokens_ans  에 저장
# your code here
from nltk.tokenize import sent_tokenize
# your code
# 이후에는 raw_data_ans 및 raw_data_ques를 사용하여 진행하십시오.

In [30]:
# 질문과 답변 매칭한 딕셔너리 - ques_and_pairs생성한 후 상위 5개 출력해 보기
# your code here

#### 1.2.4 단어 토큰화

In [31]:
# raw_data_ques 와  raw_data_ans 단어 토큰화하고 출력하기 : word_tokens_ques 저장
# your code here



#### 1.2.5 표제어 추출
WordNetleMmatizer를 사용하여 단어 토큰에서 표제어를 추출하는 함수 작성

In [32]:
# LemTokens() 정의하기 - 이전 예제 참조
# your code here

#### 1.2.6 정규화
지식 기반에 유용하지 않은 구두점을 제거하는 함수를 작성

In [33]:
# LemNormalize() 정의하기 - 이전 예제 참조
# your code here


## 2. 챗봇 기능 추가 - 코사인 유사성
데이터 세트를 문서 벡터로 변환 할 것입니다. 코사인 유사성은 유사한 벡터를 찾을 수 있게 해주며, 이러한 벡터는 의미가 비슷하다고 가정할 것입니다.

In [34]:
from nltk.tokenize import sent_tokenize
# your code

In [ ]:
# 2.1.1 입력 및 응답 목록 작성
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [ ]:
# 인사말을 수신하고 반환하는 함수 만들기
def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

In [ ]:
# 2.1.2 끝인삿말 대응하기
GOODBYE_INPUTS = ["bye", "see you!", "unit", "exit"]
GOODBYE_RESPONSES = ["Goodbye!", "See you later!", "Take care!", "Farewell!"]

In [ ]:
# 끝인삿말을 수신하고 반환하는 함수 만들기
def goodbye(sentence):
    for word in sentence.split():
        if word.lower() in GOODBYE_INPUTS:
            return random.choice(GOODBYE_RESPONSES)
    return None

In [37]:

# flag는 보통 프로그램 내에서 논리적인 상태를 나타내기 위해 사용되는 변수

flag=True # flag 변수는 챗봇 루프를 제어, True 이므로 제어상태 
print("ROBO: My name is HotelGuideBot. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while(flag==True): # flag 변수가 True인지 확인하는 조건, 
    # flag 변수의 현재 상태를 확인하는 것이며, flag가 True인 경우에 해당 조건은 참(True)

    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            
            # flag 변수의 상태가 False로 변경, 주로 루프나 조건문 내에서 특정 조건을 충족했을 때 
            # 루프를 종료하거나 조건을 거짓으로 만들어 프로그램의 실행을 제어하는 데 사용
            # 즉 user가 'thanks' 나, 'thank you'라고 입력했을 때에는 제어가 풀리게 됨
            # "ROBO: You are welcome.."가 출력되며 대화종료(user_input 종료)
            flag=False
            
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
                # tell_time 함수를 작성했으면 아래의 문장을 주석해제하세요.
#               #if(tell_time(user_response)!=None):
#                print("ROBO: "+tell_time(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is HotelGuideBot. I will answer your queries about Chatbots. If you want to exit, type Bye!
thanks
ROBO: You are welcome..


In [36]:
# # # 2.1.2 끝인삿말 대응하기
# GOODBYE_INPUTS = ["bye", "see you!", "unit", "exit"]
# GOODBYE_RESPONSES = ["Goodbye!", "See you later!", "Take care!", "Farewell!"]

# # 끝인삿말을 수신하고 반환하는 함수 만들기
# def goodbye(sentence):
#     for word in sentence.split():
#         if word.lower() in GOODBYE_INPUTS:
#             return random.choice(GOODBYE_RESPONSES)
#     return None

In [60]:
# flag=True
# print("ROBO: My name is HotelGuideBot. I will answer your queries about Chatbots. If you want to exit, type Bye!")
# while(flag==True):
#     user_response = input()
#     user_response=user_response.lower()
#     if(user_response!='bye'):
#         if(user_response=='thanks' or user_response=='thank you' ):
#             flag=False
#             print("ROBO: You are welcome..")
#         else:
#             if(greeting(user_response)!=None):
#                 print("ROBO: "+greeting(user_response))
#                 tell_time 함수를 작성했으면 아래의 문장을 주석해제하세요.
#              if(tell_time(user_response)!=None):
#                  print("ROBO: "+tell_time(user_response))
#             else:
#                 print("ROBO: ",end="")
#                 print(response(user_response))
#                 sent_tokens.remove(user_response)
#     else:
#         flag=False
#         print("ROBO: Bye! take care..")

ROBO: My name is HotelGuideBot. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: hi
bye
ROBO: Bye! take care..


챗봇의 기능은 챗봇을 실행하기 위한 루프를 만듦으로써 이루어집니다. 아래 기능을 살펴봅니다. 함수의 각 줄은 중요한 단계를 수행하기 위해 다른 함수를 호출하기 때문에 중요합니다. 'response' 함수는 챗봇이 어떻게 행동하는지에 대한 작동 방식을 담당합니다.

In [17]:
# 2.1.3 질문을 받고 답변을 반환하는 함수 만들기
# your code here
def answer(sentence1):
    for word in sentence1.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in QUESTIONS_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(QUESTIONS_RESPONSES) # Greeting_Response로 답장합니다


마지막으로 챗봇 인터페이스를 만들고 이를 중심으로 페르소나를 만들어 봅시다. 우리가 챗봇을 '제인'이라고 부르고 코사인 유사성을 사용하여 질문과 유사한 FAQ를 찾아 대답하도록 합시다.

In [18]:
def response(user_response):
    
    robo_response='' # 문자열을 포함하도록 변수를 초기화
    sent_tokens.append(user_response) # sent_messages에 사용자 응답 추가
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) # tfidf 값 가져오기

    vals = cosine_similarity(tfidf[-1], tfidf) # 코사인 유사성 값 가져오기
    idx=vals.argsort()[0][-2] 
    
    flat = vals.flatten() 
    flat.sort() # 오름차순으로 정렬
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [64]:
# 2.1.4 챗봇 테스트
# your code here
# 2.1.1 입력 및 응답 목록 작성
QUESTIONS_INPUTS = ["Do you have tuxedo services?", "room service", "Is room service served 24 hours?"]
QUESTIONS_RESPONSES = ["yes, we have tuxedo services available at the reception.", "to call room service, please dial '0' using the phone in your room.", "yes, room service is available 24 hrs."]

# 인사말을 수신하고 반환하는 함수 만들기
def questions(sentence1):
    for word in sentence1.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in QUESTIONS_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(QUESTIONS_RESPONSES) # Greeting_Response로 답장합니다.


# GOODBYE_INPUTS = ["bye", "see you!", "unit", "exit"]
# GOODBYE_RESPONSES = ["Goodbye!", "See you later!", "Take care!", "Farewell!"]

# # 끝인삿말을 수신하고 반환하는 함수 만들기
# def answer(sentence2):
#     for word in sentence.split():
#         if word.lower() in GOODBYE_INPUTS:
#             return random.choice(GOODBYE_RESPONSES)
#     return None


flag=True
print("My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("JANE: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("JANE: "+greeting(user_response))
                # tell_time 함수를 작성했으면 아래의 문장을 주석해제하세요.
#             if(tell_time(user_response)!=None):
#                 print("ROBO: "+tell_time(user_response))
            else:
                print("JANE: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("JANE: Thank you for using our services, hope you have a good time.")


      

My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Do you have tuxedo services?
JANE: 

NameError: name 'sent_tokens' is not defined

In [19]:
# 2.1.4 챗봇 테스트
# your code here
# 2.1.1 입력 및 응답 목록 작성
QUESTIONS_INPUTS = ["Do you have tuxedo services?", "room service", "Is room service served 24 hours?"]
QUESTIONS_RESPONSES = ["yes, we have tuxedo services available at the reception.", "to call room service, please dial '0' using the phone in your room.", "yes, room service is available 24 hrs."]

# 인사말을 수신하고 반환하는 함수 만들기
def questions(sentence1):
    for word in sentence1.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in QUESTIONS_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(QUESTIONS_RESPONSES) # Greeting_Response로 답장합니다.


# GOODBYE_INPUTS = ["bye", "see you!", "unit", "exit"]
# GOODBYE_RESPONSES = ["Goodbye!", "See you later!", "Take care!", "Farewell!"]

# # 끝인삿말을 수신하고 반환하는 함수 만들기
# def answer(sentence2):
#     for word in sentence.split():
#         if word.lower() in GOODBYE_INPUTS:
#             return random.choice(GOODBYE_RESPONSES)
#     return None


flag=True
print("My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("JANE: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("JANE: "+greeting(user_response))
                # tell_time 함수를 작성했으면 아래의 문장을 주석해제하세요.
#             if(tell_time(user_response)!=None):
#                 print("ROBO: "+tell_time(user_response))
            else:
                print("JANE: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("JANE: Thank you for using our services, hope you have a good time.")


      

My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Do you have tuxedo services?


NameError: name 'greeting' is not defined

## 3. Doc2vec를 이용한 챗봇 기능

챗봇을 만들기 위한 한 가지 유형의 모델을 더 다룰 것입니다. 코사인 유사성 모델 알고리즘은 두 문장 사이의 유사성을 찾는데 사용됩니다. 하지만 이제 신경망을 사용해서 이 문제를 해결해 보려합니다. 살펴보도록 하겠습니다.

Doc2Vec은 기본적으로 문서에서 벡터를 생성하는 신경망 기반 모델입니다. Doc2vec을 이해하기 위해서는 word2vec도 이해해야 합니다.

#### word2vec란?
이는 삽입 단어를 생성하는 모델이며, 여기서는 텍스트의 큰 말뭉치를 입력으로 받고 일반적으로 수백 개 차원의 벡터 공간을 생성합니다. 

2013년 9월과 10월 사이에 구글의 연구팀에 의해 두 개의 논문에 소개되었습니다. Word2Vec의 기본 가정은 유사한 맥락을 공유하는 두 단어가 유사한 의미를 공유하며 결과적으로 모델에서 유사한 벡터 표현을 공유한다는 것입니다.

예를 들어, "은행", "화폐", "계좌"는 "달러", "대출", "신용"과 같은 유사한 주변 단어와 함께 종종 사용되며, 따라서 Word2Vec에 따르면 이들은 유사한 벡터 표현을 공유합니다.

![Image](img1.png)

#### doc2vec 란?

doc2vec의 목적은 말뭉치의 모든 단어에 대한 특징 벡터를 계산하는 word2vec와 달리 문장/단락/문서의 수치 표현을 생성하는 것입니다. doc2vec은 말뭉치의 모든 문서에 대한 특징 벡터를 계산합니다. doc2vec에 의해 생성된 벡터는 문장/단락/문서 간의 유사성 찾기와 같은 작업에 사용될 수 있습니다.

<strong> doc2vec의 속성을 사용하여 우리만의 유사성 모델을 만들 것입니다.</strong>


1. Word2Vec (Word Embeddings):

- Word2Vec은 단어의 분산 표현을 학습하는 방법입니다. 
- 단어를 고정된 길이의 벡터로 표현하며, 각 벡터의 차원은 단어 사전 크기에 따라 정의됩니다.
- Word2Vec 모델은 주변 단어와의 관계를 학습하여, 유사한 문맥에서 나타나는 단어들이 비슷한 벡터 표현을 가지도록 합니다. 
- 이것은 단어 간의 의미적 유사성을 캡처하는 데 도움이 됩니다.
- Word2Vec 모델은 주로 "CBOW (Continuous Bag of Words)" 및 "Skip-gram" 두 가지 주요 아키텍처로 구현됩니다.

2.Doc2Vec (Document Embeddings):

- Doc2Vec은 문서의 분산 표현을 학습하는 방법으로, Word2Vec을 확장한 것으로 볼 수 있습니다.
- 단어뿐만 아니라 문서에 대한 벡터 표현을 생성합니다.
- Doc2Vec 모델은 문맥과 함께 문서의 벡터 표현을 학습합니다. 
- 따라서 각 문서에 고유한 벡터가 할당되고, 이 벡터는 문서의 의미와 내용을 잘 나타냅니다.
- "Distributed Memory (DM)" 및 "Distributed Bag of Words (DBOW)" 두 가지 메서드를 사용하여 문서 벡터를 학습할 수 있습니다.

주요 차이점:

- Word2Vec은 단어 수준의 분산 표현을 학습하는 반면, Doc2Vec은 문서 수준의 분산 표현을 학습합니다.
- Word2Vec은 단어 간의 의미적 유사성을 캡처하는 데 사용되며, 단어 간의 관계를 파악합니다. 

반면, 
- Doc2Vec은 문서 간의 유사성을 파악하는 데 사용됩니다.
- Word2Vec은 단어 간의 벡터 유사성을 계산하고, 단어 임베딩을 사용하여 자연어 처리 작업에 도움을 줍니다. 
- Doc2Vec은 문서 간의 유사성을 계산하고, 문서 임베딩을 사용하여 문서 분류, 검색 및 추천과 같은 작업에 활용됩니다.

관련 라이브러리를 가져오는 것으로 시작하겠습니다.

In [9]:
# subprocess와 sys 모듈을 사용하여 gensim 버전 3.8.1을 설치하는 부분입니다.
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [ ]:
# # 추가로 gensim upgrade 실행하기
# !pip install --upgrade gensim

In [10]:
# gensim의 Doc2Vec와 TaggedDocument 클래스를 가져오고,
# nltk.tokenize에서 word_tokenize 함수를 가져옵니다.
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

첫 번째 주요 목표는 데이터에 태그를 지정하는 것입니다. doc2vec 모델은 데이터를 효과적으로 사용하기 위해 태그를 지정해야 합니다. 다음은 doc2vec에 대한 [시작 코드](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) 좋은 학습 링크입니다. 다음 [링크](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) 도 유용하게 사용할 수 있으며 읽을 것을 권장합니다.

In [11]:
# 문장들을 소문자로 변환하고 단어로 토큰화하여 TaggedDocument 형식으로 변환한 후, tagged_data 리스트에 저장합니다.
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens_ques)]

In [12]:
# 최대 에포크 수를 100으로 설정합니다.
# 벡터의 크기를 20으로 설정합니다.
# 학습률을 0.025로 설정합니다.
max_epochs = 100
vec_size = 20 # 벡터가 더 크려면 이것을 증가시키세요. 이것은 더 많은 차이를 의미합니다. 
alpha = 0.025

다음 단계는 모델을 훈련시키는 것입니다. 이전처럼 훈련 과정을 실행하기 위해 `model.train` 함수를 사용하게 될 것입니다. Doc2vec 모델을 훈련하는 방법에 대한 자세한 정보는 [문서](https://radimrehurek.com/gensim/models/doc2vec.html) 를 참조하십시오.

In [13]:
# Doc2Vec 모델을 생성하고, 필요한 매개변수를 설정합니다.
model = Doc2Vec(size =vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

# model에 대한 단어 사전을 tagged_data를 사용하여 빌드합니다.
model.build_vocab(tagged_data)

# 주어진 최대 에포크 수에 대해 모델을 학습시킵니다.
for epoch in range(max_epochs):
    # 매 에포크마다 학습 상태를 출력합니다.
    print('iteration {0}'.format(epoch))
    # tagged_data를 사용하여 모델을 학습시킵니다.
    model.train(tagged_data, # 여기에 사용할 태그가 지정된 데이터
                total_examples=model.corpus_count,
                epochs=100)
    # 학습 비율을 줄입니다
    model.alpha -= 0.0002
    # 학습 비율을 고정합니다.
    model.min_alpha = model.alpha

# 학습된 모델을 "d2v.model"이라는 파일로 저장합니다. 
model.save("d2v.model")
print("Model Saved")


C:\Users\isfs0\anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### doc2vec 모델 평가

In [14]:
# 저장된 모델을 로드합니다.
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("d2v.model")
model.save("doc2vec.bin")

In [15]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

`model.infer_vector` 함수를 사용하여 문서와 관련된 벡터를 추론할 수 있습니다. 그런 다음`most_similar` 함수를 사용하여 우리가 만든 벡터와 가장 유사한 벡터를 찾을 수 있습니다. 결과는 어떻습니까?

In [16]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-0.04957128 -0.13181715  0.25946414  0.17077254 -0.14809093 -0.5838289
 -0.24057963 -0.14175332 -0.11590726 -0.1257511   0.5446048   0.3297008
  0.29079053 -0.10135418 -0.1728732  -0.3044911  -0.46978793 -0.11950009
 -0.17114438 -0.49083513]


In [17]:
# v1과 가장 유사한 문서 4개를 찾아 출력합니다.
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('30', 0.6969374418258667), ('10', 0.6017425060272217), ('0', 0.5950414538383484), ('31', 0.5860259532928467)]


In [18]:
# 가장 유사한 문서의 인덱스를 얻고, 해당 인덱스에 해당하는 문서를 출력합니다.
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]

TaggedDocument(words=['are', 'the', 'rates', 'per', 'person', 'or', 'per', 'room', '?'], tags=['30'])

이전 코드 블록의 출력에서 이 모델이 생각했던 것 만큼 효과적이지 않다는 것이 분명합니다. 이 모델이 성공하기를 기대했지만 실패한 이유가 있습니까?

자세한 내용을 보려면 이 [링크](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) 를 클릭하십시오. 이 문제의 요지는 다음과 같습니다.

> Doc2Vec는 장난감 크기(toy-size)의 데이터셋에서 좋은 결과를 얻을 수 없으므로, 더 많은 데이터를 사용하기 전까지는 의미 있는 결과를 기대해서는 안 됩니다.

### 사전 학습된 doc2vec 사용

따라서 인터넷에서 다운로드한 사전 검증된 모델을 사용하여 사용 사례를 최적화해 보겠습니다. The 우리가 취득 한 모델은 관련 언론 뉴스에 대한 훈련을 받았으며  [여기](https://github.com/jhlau/doc2vec) 서 다운로드 할 수 있습니다.

모델을 로드하고 다시 평가해 보겠습니다.

In [ ]:
# 현재 작업 디렉토리에서 "doc2vec.bin" 파일을 로드합니다.
model= Doc2Vec.load(os.getcwd()+r"//doc2vec.bin")

In [ ]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

In [ ]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

In [ ]:
# sent_tokens에 있는 모든 문장에 대해 벡터를 추론하고, 각 문장과 v1 사이의 코사인 유사도를 출력합니다.
for i in sent_tokens_ques:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

In [ ]:
# 주어진 질문에 대한 답변을 계산하는 함수 calc_prob를 정의합니다.

def calc_prob(v1, q):
    probs = dict()
    for i in q:
        # 각 질문에 대해 벡터를 추론하고, v1과의 코사인 유사도를 계산합니다.
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    # 유사도가 가장 높은 답변을 반환합니다.
    return list(sorted_d.items())[0]

In [ ]:
calc_prob(v1, sent_tokens_ques)

이 모델이 더 효과적인 것처럼 보이기 때문에, 우리의 챗봇에 통합해 봅시다.

In [ ]:
# 사용자와 대화하는 부분입니다.

flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")

while(flag==True):
    # 사용자의 입력을 받고, 소문자로 변환합니다.
    user_response = input()
    user_response=user_response.lower()
    # 사용자가 'bye'를 입력하기 전까지 다음 동작을 반복합니다:
    if(user_response!='bye'):
        # 사용자가 'thanks' 또는 'thank you'를 입력하면 대화를 종료하고 "You are welcome.."을 출력합니다.
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            # 사용자의 인사말에 대한 응답이 있는 경우 해당 응답을 출력합니다.
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            # 그렇지 않은 경우, 주어진 질문에 대한 답변을 계산하고 출력합니다.
            else:
                print("Jane: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens_ques)
                print(ques_and_pairs[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Jane: Bye! take care..")

**두 모델의 성능** 을 관찰한 후 다음과 같은 몇 가지 명확한 결론을 내릴 수 있습니다:

1. Doc2vec 모델은 단어 간의 관계를 이해하기 위해 더 많은 데이터가 필요합니다. 그리고 사전 훈련된 모델을 사용한 후에도 코사인 유사성 모델에 비해 모델의 응답의 품질이 아직 부족합니다.
2. 코사인 유사성 모델은 더 작고 잘 정의된 데이터 세트에서 더 잘 작동합니다. 이는 몇 가지 간단한 질문을 효과적으로 해결할 수 있지만 컨텍스트를 필요로 하는 복잡한 질문은 해결할 수 없다는 의미입니다.